### Questions

### Objectives
YWBAT
- Download multiple pictures from google search images using terminal
    - ```pbpaste | awk '{print substr($0, 1, length($0)-1) ">>" NR ".jpg"}' | bash```
- Implement Mongodb training and testing generators

### High level overview
- Filters: A Matrix that gets convolved over images (3x3, 5x5, etc)
     - Purpose: To find patterns
 - CNNs: Use gradient descent to find filters
     - Usually the first filter (layer) of a CNN finds EDGES
     - 2nd (filter) usually starts finding shapes within EDGES
 - Max Pooling - Is taking the max value of a filter on a layer and turning those pixels into that value. 
 - Padding - Creates equal opportunity for each pixel

Directory structure from my home
```
|-batproject
    |-train
        |-batman
        |-bat
    |-test
        |-batman
        |-bat
    |-validation
        |-batman
        |-bat
```

### Outline

In [8]:
import uuid
import requests
import glob

import pandas as pd
import numpy as np

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras import layers
from keras import Sequential
from keras import optimizers
from keras import models

import matplotlib.pyplot as plt

In [2]:
# Use glob to get your files and do a count
# this is just for a sanity check
batman_train_images = glob.glob("/Users/rafael/batproject/train/batman/*.jpg")
batman_all_images = glob.glob("/Users/rafael/batproject/*/batman/*.jpg")
bat_train_images = glob.glob("/Users/rafael/batproject/train/*/*.jpg")
len(batman_train_images), len(batman_all_images), len(bat_train_images)

(478, 576, 760)

In [13]:
train_gen = ImageDataGenerator(rescale=1./225)
test_gen = ImageDataGenerator(rescale=1./225)
validation_gen = ImageDataGenerator(rescale=1./225)

In [4]:
vars(train_gen)

{'featurewise_center': False,
 'samplewise_center': False,
 'featurewise_std_normalization': False,
 'samplewise_std_normalization': False,
 'zca_whitening': False,
 'zca_epsilon': 1e-06,
 'rotation_range': 0,
 'width_shift_range': 0.0,
 'height_shift_range': 0.0,
 'shear_range': 0.0,
 'zoom_range': [1.0, 1.0],
 'channel_shift_range': 0.0,
 'fill_mode': 'nearest',
 'cval': 0.0,
 'horizontal_flip': False,
 'vertical_flip': False,
 'rescale': 0.0044444444444444444,
 'preprocessing_function': None,
 'dtype': 'float32',
 'interpolation_order': 1,
 'data_format': 'channels_last',
 'channel_axis': 3,
 'row_axis': 1,
 'col_axis': 2,
 '_validation_split': 0.0,
 'mean': None,
 'std': None,
 'principal_components': None,
 'brightness_range': None}

In [34]:
train_gen.flow_from_directory(directory='/Users/rafael/batproject/train/', 
                              batch_size=32,
                              class_mode="categorical",
                              shuffle=True,
                              target_size=(150, 150),
                              seed=42)

validation_gen.flow_from_directory(directory='/Users/rafael/batproject/validation/', 
                                   batch_size=10,
                                   class_mode="categorical",
                                   shuffle=True,
                                   target_size=(150, 150),
                                   seed=42)

test_gen.flow_from_directory(directory='/Users/rafael/batproject/test/', 
                              batch_size=1,
                              class_mode="categorical",
                              shuffle=True,
                              target_size=(150, 150),
                              seed=42)



# This is an iterator object that will only pass in valid image files
def my_gen(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except:
            pass


Found 760 images belonging to 2 classes.
Found 32 images belonging to 2 classes.
Found 164 images belonging to 2 classes.


In [40]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [42]:
history = model.fit_generator(my_gen(train_gen),
                              steps_per_epoch=100,
                              epochs=30,
                              validation_data=my_gen(validation_gen), 
                              validation_steps=5,
                              verbose=1)

Epoch 1/30


KeyboardInterrupt: 

# debugging

In [32]:
type(train_gen.flow_from_directory("/Users/rafael/batproject/train/"))

Found 760 images belonging to 2 classes.


keras_preprocessing.image.directory_iterator.DirectoryIterator

In [35]:
next(my_gen(train_gen.flow_from_directory("/Users/rafael/batproject/train/")))

Found 760 images belonging to 2 classes.


Exception ignored in: <generator object my_gen at 0xb31bf4a98>
RuntimeError: generator ignored GeneratorExit


(array([[[[1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          ...,
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ]],
 
         [[1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          ...,
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ]],
 
         [[1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          ...,
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ],
          [1.1333333 , 1.1333333 , 1.1333333 ]],
 
         ...,
 
         [[1.1333333 , 1.1333333 , 1.1333333 ],
          [1.13333

### Assessment
- `yield` - it functions like a return but stores last step
- filters - how they work 
- workflow of creating a CNN in Keras


### Still Need
- to develop an intuition behind constructing Deep Learning Models
- 